In [1]:
#Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
#Pairing cities with our random coordinates
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [6]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters. (In the code block, we have initialized the counters at 1 because we want the first iteration of the logging for each recorded response and the set to start at 1.)
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.(endpart is formatting)
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
    # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Weather Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kapaa
Processing Record 2 of Set 1 | ushuaia
Processing Record 3 of Set 1 | port keats
Processing Record 4 of Set 1 | jamestown
Processing Record 5 of Set 1 | port elizabeth
Processing Record 6 of Set 1 | mataura
Processing Record 7 of Set 1 | ewa beach
Processing Record 8 of Set 1 | albany
Processing Record 9 of Set 1 | inuvik
Processing Record 10 of Set 1 | ostrovnoy
Processing Record 11 of Set 1 | hilo
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | charlestown
Processing Record 14 of Set 1 | cidreira
Processing Record 15 of Set 1 | vila franca do campo
Processing Record 16 of Set 1 | kogon
Processing Record 17 of Set 1 | shimoda
Processing Record 18 of Set 1 | acapulco
Processing Record 19 of Set 1 | omboue
Processing Record 20 of Set 1 | goderich
Processing Record 21 of Set 1 | narsaq
Processing Record 22 of Set 1 | casa grande
Processing Record 23 of Set 1 | khatanga

In [8]:
len(city_data)

713

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,Kapaa,US,22.0752,-159.3190,84.18,85,90,3.00,moderate rain
1,Ushuaia,AR,-54.8000,-68.3000,37.06,75,90,9.22,overcast clouds
2,Port Keats,AU,-14.2500,129.5500,88.11,30,53,6.13,broken clouds
3,Jamestown,US,42.0970,-79.2353,67.32,68,1,1.01,clear sky
4,Port Elizabeth,ZA,-33.9180,25.5701,41.29,87,0,0.00,clear sky
5,Mataura,NZ,-46.1927,168.8643,56.48,67,14,4.12,few clouds
6,Ewa Beach,US,21.3156,-158.0072,87.75,58,75,24.00,broken clouds
7,Albany,US,42.6001,-73.9662,64.92,86,0,4.12,clear sky
8,Inuvik,CA,68.3499,-133.7218,67.06,77,75,8.05,broken clouds
9,Ostrovnoy,RU,68.0531,39.5131,43.29,95,100,25.32,overcast clouds


In [11]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")